In [3]:
import pandas as pd
import numpy as np
from loaders import *

## Eyeriss for an example layer

In [4]:
show_config(ConfigRegistry.EYERISS_ARCH)
show_config(ConfigRegistry.EYERISS_COMPONENTS_DIR)
result = run_accelergy(ConfigRegistry.EYERISS_ARCH, ConfigRegistry.EYERISS_COMPONENTS_DIR)
print(result.ert)

architecture:
  # ============================================================
  # Architecture Description
  # ============================================================
  version: 0.3
  subtree:
    - name: system
      local:
        - name: DRAM
          class: DRAM
          attributes:
            type: LPDDR4
            width: 64
            block-size: 4
            word-bits: 16
      subtree:
        - name: eyeriss
          attributes:
            technology: 45nm
          local:
            - name: shared_glb
              class: smartbuffer_SRAM
              attributes:
                memory_depth: 16384
                memory_width: 64
                n_banks: 32
                block-size: 4
                word-bits: 16
                read_bandwidth: 16
                write_bandwidth: 16
            - name: DummyBuffer[0..13] # for better mapping
              class: regfile
              attributes:
                depth: 16
                width: 16
        

### VGG02_layer5


In [5]:
example_layer_stats, example_layer_mapping = run_timeloop_mapper(
    ConfigRegistry.EYERISS_ARCH, ConfigRegistry.EYERISS_COMPONENTS_DIR,
    ConfigRegistry.EYERISS_ARCH_CONSTRAINTS,
    ConfigRegistry.EYERISS_MAP_CONSTRAINTS,
    ConfigRegistry.VGG02_layer5, ConfigRegistry.DEFAULT_MAPPER_SETTING
)

input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 120
  Factorization options along problem dimension M = 495
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization o

[  0] Utilization = 0.07 | pJ/Compute =  139.321 | L5[WIO] Q56 M4 - L4[IO] M16 P56 C128 M2X - L3[] Q1 M2Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  4] Utilization = 0.57 | pJ/Compute =   10.807 | L5[WIO] Q28 M4 C128 - L4[IO] M2 P56 M4X Q2X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M2 
[  5] Utilization = 0.14 | pJ/Compute =   37.150 | L5[WIO] Q7 M64 C8 - L4[IO] M2 P56 C16 Q8X - L3[] Q1 S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M2 
[  6] Utilization = 0.19 | pJ/Compute =   49.055 | L5[WIO] Q14 M8 C16 - L4[IO] M16 P56 C4 M2X Q4X - L3[] Q1 C2Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  1] Utilization = 0.19 | pJ/Compute =   37.155 | L5[WIO] Q7 M64 C8 - L4[IO] M2 P56 C4 Q8X - L3[] Q1 C2Y S3Y - L2[I] Q1 - L1[W] R3 C2 - L0[O] M2 
[  3] Utilization = 0.57 | pJ/Compute =   23.291 | L5[WIO] Q7 M4 C4 - L4[IO] M2 P56 C32 Q8X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M8 
[  2] Utilization = 0.19 | pJ/Compute =   37.164 | L5[WIO] Q7 M64 C8 - L4[IO] M2 P56 C4 Q8X - L3[] Q1 C4Y S3Y - L2[I] Q1 - L1[W

[  4] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  0] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  2] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  7] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  5] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.




Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Compute =    5.811


[  3] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.


In [7]:
print(example_layer_mapping)


DRAM [ Weights:294912 (294912) Inputs:430592 (430592) Outputs:802816 (802816) ] 
-------------------------------------------------------------------------------
| for Q in [0:4)
|   for M in [0:4)
|     for C in [0:8)

shared_glb [ Inputs:14848 (14848) Outputs:50176 (50176) ] 
---------------------------------------------------------
|       for M in [0:4)
|         for P in [0:56)
|           for Q in [0:14) (Spatial-X)

DummyBuffer [ ] 
---------------
|             for Q in [0:1)
|               for C in [0:4) (Spatial-Y)
|                 for S in [0:3) (Spatial-Y)

ifmap_spad [ Inputs:12 (12) ] 
-----------------------------
|                   for Q in [0:1)

weights_spad [ Weights:192 (192) ] 
----------------------------------
|                     for R in [0:3)
|                       for C in [0:4)

psum_spad [ Outputs:16 (16) ] 
-----------------------------
|                         for M in [0:16)




In [8]:
print(example_layer_stats)

Buffer and Arithmetic Levels
----------------------------
Level 0
-------
=== mac ===

    SPECS
    -----
    Word bits             : 16
    Instances             : 168 (14*12)
    Compute energy        : 2.20 pJ

    STATS
    -----
    Utilized instances      : 168
    Computes (total)        : 924844032
    Cycles                  : 5505024
    Energy (total)          : 2034980565.81 pJ
    Area (total)            : 208236.00 um^2

Level 1
-------
=== psum_spad ===

    SPECS
    -----
        Technology                  : SRAM
        Size                        : 16
        Word bits                   : 16
        Block size                  : 1
        Cluster size                : 1
        Instances                   : 168 (14*12)
        Shared bandwidth            : -
        Read bandwidth              : 2.00
        Write bandwidth             : 2.00
        Multiple buffering          : 1.00
        Effective size              : 16
        Min utilization             : 0.